# 데이터 수집 자동화

In [ ]:
import streamlit as st
from datetime import datetime, timedelta
import requests
import yfinance as yf
import json
import pandas as pd
import numpy as np
from fredapi import Fred
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import itertools
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
import plotly.graph_objects as go
from collections import Counter
import collections
from st_on_hover_tabs import on_hover_tabs

## 데이터수집
start_date = '20190101'

# 한국은행 api
def fetch_data(stat_code, cycle, item_code1, item_code2, start_date= start_date):
    # 오늘 날짜 구하기 (YYYYMMDD 형식)
    today_date = datetime.today().strftime('%Y%m%d')

    # 호출 url 작성
    url = f'http://ecos.bok.or.kr/api/StatisticSearch/P4JHAI59YVHQEHBFFOTX/json/kr/1/10000/{stat_code}/{cycle}/{start_date}/{today_date}/{item_code1}/{item_code2}/'

    # 요청하고 자동 응답을 받는데 필요한 함수 호출
    response = requests.get(url)
    data = response.text

    # json-> dict자료형
    api_dict = json.loads(data)

    mid = api_dict['StatisticSearch']['row']

    # 데이터 추출
    extracted_data = [{'DATE': item['TIME'], item['ITEM_NAME1']: item['DATA_VALUE']} for item in mid]

    # 데이터프레임 생성
    df = pd.DataFrame(data=extracted_data)

    # 'DATE' 컬럼의 데이터 타입을 datetime64[ns]로 변환
    df['DATE'] = pd.to_datetime(df['DATE'])

    return df


# 환율
df_usd_krw = fetch_data('731Y001', 'D', '0000001', '원 ')

# EUR/KRW
jisu_start_date = datetime.strptime(start_date, '%Y%m%d').strftime('%Y-%m-%d')
jisu_today_date= datetime.today().strftime('%Y-%m-%d')
ticker_symbol_eur_krw = 'EURKRW=X'
eur_krw_data = yf.download(ticker_symbol_eur_krw, start=jisu_start_date, end=jisu_today_date)['Close']
df_eur_krw = pd.DataFrame({'DATE':eur_krw_data.index, 'EUR_KRW':eur_krw_data.values})

# GBP/KRW
ticker_symbol_gbp_krw = 'GBPKRW=X'
gbp_krw_data = yf.download(ticker_symbol_gbp_krw, start=jisu_start_date, end=jisu_today_date)['Close']
df_gbp_krw = pd.DataFrame({'DATE':gbp_krw_data.index, 'GBP_KRW':gbp_krw_data.values})

# JPY/KRW
ticker_symbol_jpy_krw = 'JPYKRW=X'
jpy_krw_data = yf.download(ticker_symbol_jpy_krw, start=jisu_start_date, end=jisu_today_date)['Close']
df_jpy_krw = pd.DataFrame({'DATE':jpy_krw_data.index, 'JPY_KRW':jpy_krw_data.values})

# EUR/USD
ticker_symbol_eur_usd = 'EURUSD=X'
eur_usd_data = yf.download(ticker_symbol_eur_usd, start=jisu_start_date, end=jisu_today_date)['Close']
df_eur_usd = pd.DataFrame({'DATE':eur_usd_data.index, 'EUR_USD':eur_usd_data.values})

# GBP/USD
ticker_symbol_gbp_usd = 'GBPUSD=X'
gbp_usd_data = yf.download(ticker_symbol_gbp_usd, start=jisu_start_date, end=jisu_today_date)['Close']
df_gbp_usd = pd.DataFrame({'DATE':gbp_usd_data.index, 'GBP_USD':gbp_usd_data.values})

# JPY/USD
ticker_symbol_jpy = 'JPYUSD=X'
jpy_data = yf.download(ticker_symbol_jpy, start=jisu_start_date, end=jisu_today_date)['Close']
df_jpy = pd.DataFrame({'DATE':jpy_data.index, 'JPY_USD':jpy_data.values})

# 코스피
df_kospi = fetch_data('802Y001', 'D', '0001000', '')

# 다우존스 지수
ticker_symbol_djia = '^DJI'
djia_data = yf.download(ticker_symbol_djia, start=jisu_start_date, end=jisu_today_date)['Close']
df_djia = pd.DataFrame({'DATE': djia_data.index, '다우존스': djia_data.values})

# S&P500
ticker_symbol_sp500 = '^GSPC'
sp500_data = yf.download(ticker_symbol_sp500, start=jisu_start_date, end=jisu_today_date)['Close']
df_sp500 = pd.DataFrame({'DATE': sp500_data.index, 'S&P500': sp500_data.values})

# 나스닥 지수
ticker_symbol_nasdaq = '^IXIC'
nasdaq_data = yf.download(ticker_symbol_nasdaq, start=jisu_start_date, end=jisu_today_date)['Close']
df_nasdaq = pd.DataFrame({'DATE': nasdaq_data.index, '나스닥': nasdaq_data.values})

# FTSE 100
ticker_symbol_ftse = '^FTSE'
ftse_data = yf.download(ticker_symbol_ftse, start=jisu_start_date, end=jisu_today_date)['Close']
df_ftse = pd.DataFrame({'DATE': ftse_data.index, 'FTSE': ftse_data.values})

# DAX
ticker_symbol_dax = '^GDAXI'
dax_data = yf.download(ticker_symbol_dax, start=jisu_start_date, end=jisu_today_date)['Close']
df_dax = pd.DataFrame({'DATE': dax_data.index, 'DAX': dax_data.values})

# CAC
ticker_symbol_cac = 'CAC'
cac_data = yf.download(ticker_symbol_cac, start=jisu_start_date, end=jisu_today_date)['Close']
df_cac = pd.DataFrame({'DATE': cac_data.index, 'CAC': cac_data.values})

# Nikkei
start_date = '2019/01/01'
fred = Fred(api_key='6abc0a218c6e24d76c4cc78c070e4a16')
data_nikkei = fred.get_series('NIKKEI225',observation_start = start_date)
df_Nikkei = pd.DataFrame({'DATE': data_nikkei.index, 'Nikkei': data_nikkei.values})

# 미국달러지수
ticker_symbol_dxy = 'DX-Y.NYB'
dxy_data = yf.download(ticker_symbol_dxy, start=jisu_start_date, end=jisu_today_date)['Close']
df_dxy = pd.DataFrame({'DATE': dxy_data.index, '미국달러지수': dxy_data.values})

# 6개월
data_6M = fred.get_series('DGS6MO',observation_start = start_date)
df_6M = pd.DataFrame({'DATE': data_6M.index, '미국국채금리6개월': data_6M.values})

# 1년
data_1 = fred.get_series('DGS1',observation_start = start_date)
df_1 = pd.DataFrame({'DATE': data_1.index, '미국국채금리1년': data_1.values})

# 3년
data_3 = fred.get_series('DGS3',observation_start = start_date)
df_3 = pd.DataFrame({'DATE': data_3.index, '미국국채금리3년': data_3.values})

# 5년
data_5 = fred.get_series('DGS5',observation_start = start_date)
df_5 = pd.DataFrame({'DATE': data_5.index, '미국국채금리5년': data_5.values})

# 10년
data_10 = fred.get_series('DGS10',observation_start = start_date)
df_10 = pd.DataFrame({'DATE': data_10.index, '미국국채금리10년': data_10.values})

# 금
ticker_symbol_gold = 'GC=F'
gold_data = yf.download(ticker_symbol_gold, start=jisu_start_date, end=jisu_today_date)['Close']
df_gold = pd.DataFrame({'DATE': gold_data.index, '금': gold_data.values})

# 은
ticker_symbol_silver = 'SI=F'
silver_data = yf.download(ticker_symbol_silver, start=jisu_start_date, end=jisu_today_date)['Close']
df_silver = pd.DataFrame({'DATE': silver_data.index, '은': silver_data.values})

# WTI 원유 가격
data_wti = fred.get_series('DCOILWTICO',observation_start = start_date)
df_wti = pd.DataFrame({'DATE': data_wti.index, 'WTI': data_wti.values})

# 천연가스
ticker_symbol_gas = 'NG=F'
gas_data = yf.download(ticker_symbol_gas, start=jisu_start_date, end=jisu_today_date)['Close']
df_gas = pd.DataFrame({'DATE': gas_data.index, '천연가스': gas_data.values})

# 원본 데이터프레임들
dfs = [df_usd_krw, df_eur_krw, df_gbp_krw, df_jpy_krw, df_eur_usd,df_gbp_usd,df_jpy, df_djia, df_sp500, df_nasdaq, df_ftse, df_dax, df_cac,df_Nikkei, df_dxy, df_6M, df_1, df_3, df_5, df_10,df_gold, df_silver, df_gas, df_wti, df_kospi]

# 데이터프레임 병합
df_combined_D = dfs[0]
for df in dfs[1:]:
    df_combined_D = pd.merge(df_combined_D, df, on='DATE', how='left')

# 날짜를 기준으로 정렬
df_combined_D.sort_values(by='DATE', inplace=True)

# nan값 채우기
df_combined_D.fillna(method='ffill',inplace = True)
df_combined_D.fillna(method='bfill',inplace = True)

# 컬럼명 바꾸기
df = df_combined_D.rename(columns={"DATE":"날짜","원/미국달러(매매기준율)":"환율"})

df.to_csv('지수.csv',index=False)

## 모델링
# 날짜 칼럼 --> datetime 형태로 변환
df['날짜'] = pd.to_datetime(df['날짜'])

# 날짜 칼럼을 인덱스로 설정
df.set_index('날짜', inplace=True)

# Feature Engineering: 이동평균 및 과거 환율 값 추가
df['환율_7일_평균'] = df['환율'].rolling(window=7).mean()

for i in range(1,8):
    df[f'환율_{i}일전'] = df['환율'].shift(i)
df.dropna(inplace=True)

# 정답 컬럼 생성
df.loc[:,'target'] = df.loc[:,'환율']

X = df.iloc[:, :-1]
y = df.loc[:,['target']]

# 정규화 진행
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# LSTM 입력 데이터 형태 변환
def make_dataset(X, y, time_steps):
    X_data_total = []
    y_data_total = []
    for start in np.arange(len(X)-time_steps):
        stop = start + time_steps
        # X_data --> 슬라이싱
        X_data = X[start:stop, :]
        X_data_total.append(X_data)
        # y_data --> 인덱싱
        y_data = y[stop]
        y_data_total.append(y_data)
    return np.array(X_data_total), np.array(y_data_total)

time_steps = 30
X_data,y_data = make_dataset(X = X_scaled, y = y_scaled, time_steps = time_steps)

# 학습용 / 평가용 데이터 생성
train_size = int(len(X_data) * 0.7)
valid_size = int(len(X_data) * 0.2)
test_size = len(X_data) - train_size - valid_size

X_train = X_data[0:train_size,:,:]
y_train = y_data[0:train_size,:]
X_valid = X_data[train_size:train_size + valid_size,:,:]
y_valid = y_data[train_size:train_size + valid_size,:]
X_test = X_data[train_size + valid_size:len(X_data),:,:]
y_test = y_data[train_size + valid_size:len(X_data),:]

# 시드 고정
seed = 45
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 최적의 파라미터
best_params = {'learning_rate': 0.001, 'batch_size': 32, 'dropout': 0.1, 'optimizer': 'adam', 'epochs': 20, 'regularization': 0.01}

# LSTM 모델 정의
tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(64, kernel_regularizer=l2(best_params['regularization']),input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, kernel_regularizer=l2(best_params['regularization'])))

optimizer = Adam(learning_rate=best_params['learning_rate'])
model.compile(optimizer=optimizer, loss='mean_squared_error')

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
                    validation_data=(X_valid, y_valid), shuffle=False, callbacks=[early_stopping])

# 예측
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)
test_predict = model.predict(X_test)

# 예측 값 정규화 이전으로 복원
train_predict = scaler_y.inverse_transform(train_predict)
valid_predict = scaler_y.inverse_transform(valid_predict)
test_predict = scaler_y.inverse_transform(test_predict)

# 원래 값 정규화 이전으로 복원
y_train_actual =  scaler_y.inverse_transform(y_train)
y_valid_actual =  scaler_y.inverse_transform(y_valid)
y_test_actual =  scaler_y.inverse_transform(y_test)

total_predict = np.concatenate([train_predict, valid_predict, test_predict])
total_actual = np.concatenate([y_train_actual, y_valid_actual, y_test_actual])

index = df.iloc[time_steps:].index
df_predict = pd.DataFrame(data = total_predict,index=index, columns=['예측환율'])
df_predict.loc[:,'실제환율'] = y.iloc[time_steps:].values.flatten()
df_predict['예측환율'] = df_predict['예측환율'].astype(float).round(2)

#주말 채우기
date_range = pd.date_range(start=df_predict.index[0], end=df_predict.index[-1], freq='D')
new = pd.DataFrame({'날짜':date_range})
df_predict = df_predict.reset_index()
df_predict = pd.merge(new,df_predict, on='날짜', how='left')
df_predict.fillna(method='ffill',inplace = True)

### 향후 30일 예측 - 일별
future_dates = pd.date_range(start = df.index[-1] + pd.Timedelta(days = 1), periods = 30, freq='B')

future_preds = []
input_data = np.array(X_scaled[-30:])

for date in future_dates:
    # input_data를 모델 입력 형태에 맞게 reshape
    input_data_reshaped = input_data.reshape(1, 30, -1)
    pred = model.predict(input_data_reshaped)
    future_preds.append(pred[0])

    # When creating new_row, ensure all features are included
    new_row = np.append(input_data[0, 1:], pred[0])
    input_data = np.append(input_data[1:], new_row.reshape(1, -1), axis=0)

future_preds = scaler_y.inverse_transform(np.array(future_preds).reshape(-1, 1))

# 소수점 둘째자리까지 반올림
future_preds = np.round(future_preds, 2)

# 미래 예측 데이터프레임
future_df = pd.DataFrame({'날짜': future_dates, '예측환율':  future_preds.flatten()})

# 주말 포함 - 주말에는 금요일 환율로 대체
dates = pd.date_range(start = df.index[-1] + pd.Timedelta(days = 1), periods = 42, freq='D')
new = pd.DataFrame({'날짜' : dates})
pred_dates_df = pd.merge(new, future_df, on = '날짜', how = 'left')
pred_dates_df.fillna(method = 'ffill',inplace = True)

# 예측 환율의 오차범위 만들기 - 예측환율_일별_최소 칼럼, 예측환율_일별_최대 칼럼 생성(소수점 둘째자리까지)
pred_dates_df['예측환율'] = pred_dates_df['예측환율'].apply(lambda x : round(x, 2))
pred_dates_df['예측환율_일별_최소'] = np.round(pred_dates_df['예측환율'] - 1.96*np.std(pred_dates_df['예측환율']), 2)
pred_dates_df['예측환율_일별_최대'] = np.round(pred_dates_df['예측환율'] + 1.96*np.std(pred_dates_df['예측환율']), 2)

df_end = pd.concat([df_predict,pred_dates_df[['날짜','예측환율']]])

pred_dates_df.to_csv("pred.csv",index=False)
df_end.to_csv("total.csv", index = False)

### 3개월 월별 예측
future_days = 60
future_month_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=future_days, freq='B')

input_data = np.array(X_scaled[-30:])
future_month_preds = []

for date in future_month_dates:
    input_data_reshaped = input_data.reshape(1, 30, -1)
    pred1 = model.predict(input_data_reshaped)
    future_month_preds.append(pred1[0])
    new_row = np.append(input_data[0, 1:], pred1[0])
    input_data = np.append(input_data[1:], new_row.reshape(1, -1), axis=0)

future_month_preds = scaler_y.inverse_transform(np.array(future_month_preds).reshape(-1, 1))
future_month_preds = np.round(future_month_preds, 2)

# 미래 예측 데이터프레임
future_month_df = pd.DataFrame({'예측날짜': future_month_dates, '예측환율': future_month_preds.flatten()})

### 주말 포함 병합
dates1 = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=future_days, freq='D')
new1 = pd.DataFrame({'예측날짜': dates1})
pred_month_df = pd.merge(new1, future_month_df, on='예측날짜', how='left')
pred_month_df = pred_month_df.fillna(method='ffill')

# 월별 마지막 예측환율 값 찾기
pred_month_df['예측날짜'] = pred_month_df['예측날짜'].dt.to_period('M')
last_day_preds = pred_month_df.groupby('예측날짜').last().reset_index()
last_day_preds.columns = ['예측날짜', '예측환율']

# 월별 마지막 예측환율 값 --> 월별 데이터로 넣기(날짜 형태 연도-월)
last_day_preds['예측날짜'] = last_day_preds['예측날짜'].dt.strftime('%Y-%m')
last_day_preds['예측환율'] = last_day_preds['예측환율'].apply(lambda x: round(x, 2))
last_day_preds['예측환율_최소'] = np.round(last_day_preds['예측환율'] - 1.96 * np.std(last_day_preds['예측환율']), 2)
last_day_preds['예측환율_최대'] = np.round(last_day_preds['예측환율'] + 1.96 * np.std(last_day_preds['예측환율']), 2)

last_day_preds.to_csv('환율예측_월별.csv',index=False)

# 웹 페이지

In [ ]:
import streamlit as st
from datetime import datetime, timedelta
import requests
import yfinance as yf
import json
import pandas as pd
import numpy as np
from fredapi import Fred
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import itertools
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
import plotly.graph_objects as go
from collections import Counter
import collections
from st_on_hover_tabs import on_hover_tabs
from telegram import Bot
import schedule
import threading
import plotly.express as px
from streamlit_lightweight_charts import renderLightweightCharts

# 페이지 레이아웃 설정
st.set_page_config(page_title="Currency Forecast",
                   page_icon="💵",
                   layout="wide")
# CSS로 여백 제거
st.markdown("""
    <style>
    .reportview-container .main .block-container{
        padding-top: 1rem;
        padding-left: 1rem;
        padding-right: 1rem;
        padding-bottom: 1rem;
    }
    .centered-text {
        text-align: center;
    }
    </style>
""", unsafe_allow_html=True)

# 제목
st.markdown("<h1 class='centered-text' style='font-size: 5em;'>환율 예측 프로그램</h1>", unsafe_allow_html=True)
st.markdown("<h2 class='centered-text update-text' style='font-size: medium; color: gray;'>오전 9:00에 업데이트 됩니다.</h2>", unsafe_allow_html=True)
st.markdown('<style>' + open('./style.css').read() + '</style>', unsafe_allow_html=True)

# 그래프 설명
col1, col2 = st.columns([10,0.5])

with col2:
    popover = st.popover(":chart_with_upwards_trend:")
    # popover.markdown("**그래프 확대 및 축소**")
    multi = '''**그래프 확대 및 축소**
    ▷ 확대 : 그래프 영역을 마우스로 드래그하여 원하는 부분을 선택하세요.
    ▷ 축소 : 더블클릭하면 원래 크기로 돌아갑니다.'''
    popover.markdown(multi)


# 사이드 바
with st.sidebar:
    tabs = on_hover_tabs(tabName=['Dashboard','Keyword', 'Information'],
                         iconName=['dashboard','format_size', 'info'], default_choice=0)

## 환율 불러오기(시가,저가,고가,종가)
# 한국은행 api
def fetch_data(stat_code, cycle, item_code1,item_code2, start_date):
    today_date = datetime.today().strftime('%Y%m%d')
    url = f'http://ecos.bok.or.kr/api/StatisticSearch/P4JHAI59YVHQEHBFFOTX/json/kr/1/10000/{stat_code}/{cycle}/{start_date}/{today_date}/{item_code1}/{item_code2}/'
    response = requests.get(url)
    data = response.text
    api_dict = json.loads(data)
    mid = api_dict['StatisticSearch']['row']
    extracted_data = [{'DATE': item['TIME'], '데이터': item['DATA_VALUE']} for item in mid]
    df = pd.DataFrame(data=extracted_data)
    return df

df_usd_krw = fetch_data('731Y001', 'D', '0000001', '','20040101')
df_open = fetch_data('731Y003', 'D', '0000002','','20040101')
df_low = fetch_data('731Y003', 'D', '0000004','','20040101')
df_high = fetch_data('731Y003', 'D', '0000005','','20040101')
df_close = fetch_data('731Y003', 'D', '0000003','','20040101')

# 날짜를 인덱스로 설정하고 데이터프레임을 병합
df_open.set_index('DATE', inplace=True)
df_low.set_index('DATE', inplace=True)
df_high.set_index('DATE', inplace=True)
df_close.set_index('DATE', inplace=True)

df_open = df_open.rename(columns={'데이터': 'Open'})
df_low = df_low.rename(columns={'데이터': 'Low'})
df_high = df_high.rename(columns={'데이터': 'High'})
df_close = df_close.rename(columns={'데이터': 'Close'})

df = df_open.join([df_low, df_high, df_close], how='inner')
df.index = pd.to_datetime(df.index,format='%Y%m%d')

# dashboard 탭
if tabs =='Dashboard':
    ## 실제환율만 있는 그래프
    # 제목
    st.title('원/달러 환율')
    # Fetch today's exchange rate
    rate = pd.read_csv('지수.csv')

    rate['환율'] = pd.to_numeric(rate['환율'])

    today_rate = rate['환율'].iloc[-1]

    # Fetch yesterday's exchange rate
    yesterday_rate = rate['환율'].iloc[-2]

    # Calculate the change
    change = today_rate - yesterday_rate
    percentage_change = (change / yesterday_rate) * 100

    st.metric(label="매매기준율", value=f"{today_rate:.2f}", delta=f"{change:.2f} ({percentage_change:.2f}%)",delta_color ='inverse')

    tab1, tab2 = st.tabs(['일반','차트'])

    with tab1:
        ## 영역 그래프
        # 영역 그래프 생성
        fig_area = px.area(df, x=df.index, y='Close')
        fig_area.update_layout(xaxis_title='날짜',yaxis_title='환율',yaxis=dict(range=[800, 1600]))

        fig_area.update_traces(hovertemplate='<b>%{x|%Y-%m-%d}</b><br>' + '시가: %{customdata[0]:.2f}<br>' + '고가: %{customdata[1]:.2f}<br>'+'저가: %{customdata[2]:.2f}<br>' +'종가: %{y:.2f}<br>',customdata=df[['Open', 'High', 'Low']].values)

        # hover 스타일 편집
        fig_area.update_layout(hoverlabel_bgcolor="white",hoverlabel_font_size=12,hoverlabel_font_family="Rockwell")

        # Streamlit에 그래프 표시
        st.plotly_chart(fig_area, use_container_width=True)

    with tab2:
        # Define colors
        COLOR_UP = 'rgba(255, 0, 0, 0.9)'   # Blue for bullish (rising) candles
        COLOR_DOWN = 'rgba(0, 0, 255, 0.9)' # Red for bearish (falling) candles

        # Fetch data function
        def fetch_data(stat_code, cycle, item_code1, item_code2, start_date):
            today_date = datetime.today().strftime('%Y%m%d')
            url = f'http://ecos.bok.or.kr/api/StatisticSearch/P4JHAI59YVHQEHBFFOTX/json/kr/1/10000/{stat_code}/{cycle}/{start_date}/{today_date}/{item_code1}/{item_code2}/'
            response = requests.get(url)
            data = response.text
            api_dict = json.loads(data)
            mid = api_dict['StatisticSearch']['row']
            extracted_data = [{'DATE': item['TIME'], '데이터': item['DATA_VALUE']} for item in mid]
            df = pd.DataFrame(data=extracted_data)
            return df

        # Fetching exchange rate data
        df_usd_krw = fetch_data('731Y001', 'D', '0000001', '', '20040101')
        df_open = fetch_data('731Y003', 'D', '0000002', '', '20040101')
        df_low = fetch_data('731Y003', 'D', '0000004', '', '20040101')
        df_high = fetch_data('731Y003', 'D', '0000005', '', '20040101')
        df_close = fetch_data('731Y003', 'D', '0000003', '', '20040101')

        # Setting DATE as index and merging dataframes
        df_open.set_index('DATE', inplace=True)
        df_low.set_index('DATE', inplace=True)
        df_high.set_index('DATE', inplace=True)
        df_close.set_index('DATE', inplace=True)

        df_open = df_open.rename(columns={'데이터': 'open'})
        df_low = df_low.rename(columns={'데이터': 'low'})
        df_high = df_high.rename(columns={'데이터': 'high'})
        df_close = df_close.rename(columns={'데이터': 'close'})

        df = pd.concat([df_open, df_low, df_high, df_close], axis=1).reset_index()
        df['datetime'] = pd.to_datetime(df['DATE'])
        df['time'] = df['datetime'].dt.strftime('%Y-%m-%d')

        # Export to JSON format
        candles = json.loads(df.filter(['time', 'open', 'high', 'low', 'close'], axis=1).to_json(orient="records"))

        chartMultipaneOptions = [
            {
                "layout": {
                    "background": {
                        "type": "solid",
                        "color": 'white'
                    },
                    "textColor": "black"
                },
                "grid": {
                    "vertLines": {
                        "color": "rgba(197, 203, 206, 0.5)"
                    },
                    "horzLines": {
                        "color": "rgba(197, 203, 206, 0.5)"
                    }
                },
                "crosshair": {
                    "mode": 0
                },
                "priceScale": {
                    "borderColor": "rgba(197, 203, 206, 0.8)",
                    "scaleMargins": {
                        "top": 0.1,
                        "bottom": 0.1
                    }
                },
                "timeScale": {
                    "borderColor": "rgba(197, 203, 206, 0.8)",
                    "barSpacing": 10,
                    "minBarSpacing": 8,
                    "timeVisible": False,
                    "secondsVisible": False
                }
            }
        ]

        seriesCandlestickChart = [
            {
                "type": 'Candlestick',
                "data": candles,
                "options": {
                    "upColor": COLOR_UP,
                    "downColor": COLOR_DOWN,
                    "borderVisible": True,
                    "wickUpColor": COLOR_UP,
                    "wickDownColor": COLOR_DOWN,
                    "borderUpColor": COLOR_UP,
                    "borderDownColor": COLOR_DOWN
                }
            }
        ]

        renderLightweightCharts([
            {
                "chart": chartMultipaneOptions[0],
                "series": seriesCandlestickChart
            }
        ], 'multipane')



    st.subheader(' ',divider='grey')

    name, pop = st.columns([10,50])
    name.title('환율 예측')

    with pop:
        popover = st.popover(":grey_question:")
        multi = '''**일별**
        일별 예측 환율은 두 가지 방식으로 제공합니다.
        첫째, 30일치 예측값을 한눈에 볼 수 있는 그래프로 제공하며, 주말에는 금요일 예측값이 표시됩니다.
        둘째, 원하는 미래 날짜를 선택하면 해당 날짜의 예측값을 즉시 확인할 수 있습니다.
        **월별**
        이후 3개월의 월별 환율을 예측한 결과값을 보여줍니다.
        각 월별 예측 환율은 해당 달의 말일 기준으로 계산되었습니다.
        **환율계산기**
        이 환율 예측 계산기는 사용자가 원하는 금액을 입력하면 예측된 환율을 사용하여 변환된 금액을 계산해줍니다.
        변환된 원화 금액은 소수점 두 번째 자리까지 표시됩니다.
        '''
        popover.markdown(multi)

    left, right = st.columns([10,5])

    with left:
        left.markdown("<h3 style='font-size: medium;'>일별</h3>", unsafe_allow_html=True)
        # CSV 파일 읽기
        pred_dates_df1 = pd.read_csv('pred.csv')

        # 데이터 프레임 설정
        pred_dates_df1['날짜'] = pd.to_datetime(pred_dates_df1['날짜'])

        # 30일일 데이터만 선택
        pred_dates_df1 = pred_dates_df1.head(30)

        # 퍼센트 변화 계산
        pred_dates_df1['변화율'] = pred_dates_df1['예측환율'].pct_change() * 100

        # 첫 번째 날 및 NaN 값 처리
        pred_dates_df1['변화율'].iloc[0] = 0

        # 화살표와 퍼센트 변화 추가
        def format_change_text(row):
            if row['변화율'] == 0 or np.isnan(row['변화율']):
                return "<span style='color: grey;'>&nbsp;-</span>"
            elif row['변화율'] > 0:
                return f"<span style='color: red;'>▲ {row['변화율']:.2f}%</span>"
            else:
                return f"<span style='color: blue;'>▼ {row['변화율']:.2f}%</span>"

        pred_dates_df1['변화_텍스트'] = pred_dates_df1.apply(format_change_text, axis=1)

        # 그래프 생성
        fig = go.Figure()

        # 예측 환율 데이터 추가
        fig.add_trace(go.Scatter(
            x=pred_dates_df1['날짜'],
            y=pred_dates_df1['예측환율'],
            mode='lines+markers',
            marker=dict(color='green', size=6),
            name='예측환율',
            line=dict(color='green'),
            hovertemplate='<b>%{x|%Y-%m-%d}</b><br>예측환율: %{y}<br><span style="text-align: center; display: block;">변화율: %{customdata}</span><extra></extra>',
            customdata=pred_dates_df1['변화_텍스트'],
            showlegend=False
        ))

        # 오차범위 추가
        fig.add_trace(go.Scatter(x=pred_dates_df1['날짜'].tolist() + pred_dates_df1['날짜'][::-1].tolist(),y=pred_dates_df1['예측환율_일별_최대'].tolist() + pred_dates_df1['예측환율_일별_최소'][::-1].tolist(),fill='toself',fillcolor='rgba(0,100,80,0.2)',line_color='rgba(255,255,255,0)',hoverinfo="skip",showlegend=False))


        # 그래프 레이아웃 설정
        fig.update_layout(xaxis_title='날짜', yaxis_title='환율', xaxis=dict(tickformat='%Y-%m-%d'))

        # hover 스타일 편집
        fig.update_layout(hoverlabel_bgcolor="white", hoverlabel_font_size=12, hoverlabel_font_family="Rockwell")

        # 그래프 표시
        left.plotly_chart(fig, use_container_width=True)

    with right:
        ## 예측 값 표시
        # CSV 파일 읽기
        pred_dates_df1 = pd.read_csv('pred.csv')

        # 데이터 프레임 설정
        pred_dates_df1['날짜'] = pd.to_datetime(pred_dates_df1['날짜'])

        # 30일 데이터만 선택
        pred_dates_df1 = pred_dates_df1.head(30)

        # 날짜 선택
        date_input = st.date_input(':date:', value=pred_dates_df1['날짜'].min(), min_value = pred_dates_df1['날짜'].min(), max_value = pred_dates_df1['날짜'].max())

        # 날짜 입력 및 예측 결과 출력
        if date_input:
            try:
                date = pd.to_datetime(date_input)
                if date in pred_dates_df1['날짜'].values:
                    row = pred_dates_df1[pred_dates_df1['날짜'] == date]
                    pred_value = row['예측환율'].values[0]
                    if date.weekday() == 5 or date.weekday() == 6:
                        st.write(':red[주말에는 환율이 표시되지 않습니다.]')
                    else:
                        st.write(f"{date_input.strftime('%Y-%m-%d')}의 예측환율: {row['예측환율'].values[0]}")
                        st.write(f"{date_input.strftime('%Y-%m-%d')}의 예측환율 범위: {row['예측환율_일별_최소'].values[0]} ~ {row['예측환율_일별_최대'].values[0]}")
                else:
                    st.write("해당 날짜가 예측범위에 없습니다.")
            except Exception as e:
                st.write(f"오류가 발생했습니다: {e}")

        st.subheader(' ')

        ## 환율 계산기
        # 환율 데이터 불러오기
        df1 = pd.read_csv('pred.csv')

        # 세션 상태 초기화
        if 'calculate_usd' not in st.session_state:
            st.session_state.calculate_usd = False
        if 'calculate_krw' not in st.session_state:
            st.session_state.calculate_krw = False

        # 팝업표시
        with st.expander('환율 계산기'):
            # 날짜 입력 위젯 (데이터셋 내의 미래 날짜 허용)
            d_today = datetime.today().date()
            d_tomorrow = d_today + timedelta(days=1)
            d_max = pd.to_datetime(df1['날짜'].head(30)).max().date()
            selected_date = st.date_input('날짜 선택', value=d_tomorrow, min_value=d_tomorrow, max_value=d_max)
            formatted_date = selected_date.strftime('%Y-%m-%d')

            # 선택된 날짜가 주말인지 확인
            is_weekend = selected_date.weekday() >= 5

            # 환율 데이터를 가져오는 함수
            def get_exchange_rate(date):
                data = df1[df1['날짜'] == date]
                if not data.empty:
                    return data['예측환율'].values[0]
                else:
                    return None

            if is_weekend:
                st.warning("주말에는 환율이 표시되지 않습니다.")
            else:
                exchange_rate = get_exchange_rate(formatted_date)

                if exchange_rate:
                    # 입력 칸 두 개 (원화/외화)
                    col1, col2 = st.columns(2)


                    # 원화 입력
                    with col1:
                        krw_amount1 = st.number_input('변환할 원화(KRW) 금액 입력', min_value=0, key='krw_amount')
                        if krw_amount1 > 0:
                            st.session_state.calculate_usd = True
                            st.session_state.calculate_krw = False
                        else:
                            st.session_state.calculate_usd = False

                    # 외화 입력
                    with col2:
                        usd_amount2 = st.number_input('변환할 외화(USD) 금액 입력', min_value=0.0, key='usd_amount')
                        if usd_amount2 > 0:
                            st.session_state.calculate_usd = False
                            st.session_state.calculate_krw = True
                        else:
                            st.session_state.calculate_krw = False

                    # 계산 수행
                    if st.session_state.calculate_usd and krw_amount1 > 0:
                        # 원화를 달러로 변환
                        usd_amount_converted = krw_amount1 / exchange_rate
                        col1, col2, col3 = st.columns([4, 2, 4])
                        with col1:
                            st.header("미국(USD)")
                            st.subheader(f'{usd_amount_converted:.2f} USD')
                        with col2:
                            st.image('arrow-left-right.svg', width=50)
                        with col3:
                            st.header("한국(KRW)")
                            st.subheader(f'{krw_amount1:,} 원')
                    elif st.session_state.calculate_krw and usd_amount2 > 0:
                        # 달러를 원화로 변환
                        krw_amount_converted = round(usd_amount2 * exchange_rate)
                        col1, col2, col3 = st.columns([4, 2, 4])
                        with col1:
                            st.header("미국(USD)")
                            st.subheader(f'{usd_amount2:.2f} USD')
                        with col2:
                            st.image('arrow-left-right.svg', width=50)
                        with col3:
                            st.header("한국(KRW)")
                            st.subheader(f'{krw_amount_converted:,} 원')
                    else:
                        st.warning("원화 또는 외화 금액을 입력하세요.")
                else:
                    st.error("선택한 날짜에 대한 환율 데이터를 찾을 수 없습니다.")
    st.markdown('---')

    left_, right_ = st.columns([10,5])
    with left_:
        left_.markdown("<h4 style='font-size: medium;'>월별</h4>", unsafe_allow_html=True)
        ## 예측환율 - 월별
        # CSV 파일 읽기
        pred_dates_month = pd.read_csv('환율예측_월별.csv')
        pred_dates_month = pred_dates_month.iloc[:3,:]

       # HTML 테이블 생성
        html_table_month = pred_dates_month.to_html(index=False, justify='center')

        # CSS를 사용하여 가운데 정렬 및 너비 조정
        st.markdown(
            """
            <style>
            .month-table {
                width: 100%;
                margin: 0 auto;
            }
            .month-table table {
                width: 100%;
                table-layout: fixed;
            }
            .month-table th, .month-table td {
                text-align: center !important;
            }
            </style>
            """,
            unsafe_allow_html=True
        )

        # HTML 테이블 출력
        st.markdown(f'<div class="month-table">{html_table_month}</div>', unsafe_allow_html=True)

    with right_:
        col1_, col2_ = right_.columns([10,1])


        # 텔레그램 알림 관련 코드
        col1_.subheader('텔레그램 알림')

        with col2_:
            popover = st.popover(":grey_question:")
            multi = ''' **QR 코드 스캔**
            - 아래 QR 코드를 스캔하여 텔레그램 대화방에 입장하세요.
            - 텔레그램 앱이 없는 경우, 먼저 앱을 다운로드하고 회원가입을 완료해주세요.
            **텔레그램 알림 설정**
            - "텔레그램 알림 받기"를 눌러 예측 환율 정보를 편리하게 휴대폰으로 받아보세요. 이 기능을 통해 다음 날의 환율 예측값을 알림으로 받고, 휴대폰에서도 손쉽게 환율 정보를 확인할 수 있습니다.
            '''
            popover.markdown(multi)

        qr, button = right_.columns(2)
        qr.image('텔레그램_qr코드.png',caption = '텔레그램 접속 QR')

        # 텔레그램 봇 토큰과 채널 ID 설정
        TELEGRAM_BOT_TOKEN = "7445529925:AAGDMvyDYyc3FNQgKa0FBrOqWBHp9DbbdBw"
        bot = Bot(token = TELEGRAM_BOT_TOKEN)
        chat_id = -1002180124685
        alarm_started = False
        scheduler_thread = None

        def send_telegram_message(chat_id, message):
            try:
                url = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage'
                payload = {
                    'chat_id': chat_id,
                    'text': message
                }
                response = requests.post(url, json=payload)
                if response.status_code != 200:
                    st.write(f"Failed to send message: {response.status_code} - {response.text}")
            except Exception as e:
                st.write(f"Error sending message: {e}")

        def daily_alarm():
            try:
                today = datetime.now()
                today_str = today.strftime('%Y-%m-%d')
                next_day = datetime.now() + timedelta(days=1)
                next_day_str = next_day.strftime('%Y-%m-%d')
                button.write(f"Preparing message for {next_day_str}")
                # 예측날짜_일별 열의 날짜 형식을 문자열로 변환하여 비교
                pred_dates_df1['날짜'] = pd.to_datetime(pred_dates_df1['날짜']).dt.strftime('%Y-%m-%d')

                if next_day_str in pred_dates_df1['날짜'].values:
                    row = pred_dates_df1[pred_dates_df1['날짜'] == next_day_str]
                    pred_value = row['예측환율'].values[0]
                    today_value = df_usd_krw.iloc[-1].values[1]

                    if date.weekday() == 5 or date.weekday() == 6:
                        send_telegram_message(chat_id, f"내일({next_day_str})은 주말이므로 환율 데이터가 표시되지 않습니다.")
                    else:
                        message = f"오늘({today_str})의 환율 : {today_value}원\n"\
                                    f"내일({next_day_str})의 예측환율 : {pred_value}\n"\
                                    f"예측환율 범위 : {row['예측환율_일별_최소'].values[0]} ~ {row['예측환율_일별_최대'].values[0]}"
                        send_telegram_message(chat_id, message)
                else:
                    send_telegram_message(chat_id, f"내일({next_day_str})의 예측환율 데이터가 없습니다.")
            except Exception as e:
                button.write(f"Error in daily_alarm: {e}")


        def start_alarm():
            global alarm_started, scheduler_thread
            if not alarm_started:
                send_telegram_message(chat_id, "알림이 시작되었습니다")
                button.write("알림이 시작되었습니다")
                schedule.every().day.at("09:40").do(daily_alarm)
                alarm_started = True
                scheduler_thread = threading.Thread(target=run_scheduler)
                scheduler_thread.start()
                daily_alarm()  # Start immediately for testing

        def stop_alarm():
            global alarm_started
            send_telegram_message(chat_id, "알림이 중단되었습니다")
            button.write("알림이 중단되었습니다")
            if alarm_started:
                schedule.clear()
                alarm_started = False

        def run_scheduler():
            while alarm_started:
                schedule.run_pending()
                time.sleep(1)

        if button.button('텔레그램 알림 받기'):
            if not alarm_started:
                start_alarm()
            else:
                button.write("이미 알림이 시작되었습니다.")


elif tabs == 'Keyword':
    ## 키워드 - 변곡점
    # 제목
    st.title('키워드 분석')

    col1, col2 = st.columns([10, 0.5])

    # CSV 파일 읽기
    df = pd.read_csv('최종.csv')
    keyword_df = pd.read_csv('환율(경제)_수정.csv')

    # 데이터 프레임 설정
    df['날짜'] = pd.to_datetime(df['날짜'])
    df.set_index('날짜', inplace=True)

    # 키워드 데이터 프레임 설정
    keyword_df['일자'] = pd.to_datetime(keyword_df['일자'])
    keyword_df.set_index('일자', inplace=True)

    # 이동 평균 계산
    df['100일 이동 평균'] = df['환율'].rolling(window=100, min_periods=1).mean()

    # 변곡점 탐지
    df['Signal'] = 0.0
    df.iloc[100:, df.columns.get_loc('Signal')] = np.where(df['환율'][100:] > df['100일 이동 평균'][100:], 1.0, 0.0)
    df['Position'] = df['Signal'].diff()

    # 변곡점 필터링 (30일 이상의 간격)
    filtered_positions = []
    last_date = df.index[0]
    for date, pos in df['Position'][df['Position'] != 0].items():
        if (date - last_date).days >= 30:
            filtered_positions.append((date, pos))
            last_date = date

    # 키워드 집계 함수
    def get_top_keywords(date, keyword_df, days=30, top_n=10):
        start_date = date - pd.Timedelta(days=days)
        relevant_keywords = keyword_df.loc[start_date:date]
        all_text = ' '.join(relevant_keywords['특성추출(가중치순 상위 50개)'].tolist())
        tokens = all_text.split(',')
        word_counts = Counter(tokens)
        top_keywords_tuples = collections.Counter(word_counts).most_common(top_n)
        top_keywords = [keyword for keyword, count in top_keywords_tuples]
        return top_keywords

    # 플로틀리 그래프 생성
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['환율'], mode='lines', name='환율', line=dict(color='gray')))

    # 변곡점 추가
    added_legends = {"상승": False, "하락": False}  # 범례 표시를 위한 플래그

    for date, pos in filtered_positions:
        top_keywords = get_top_keywords(date, keyword_df)
        hover_text = f"<br>상위 키워드:<br>" + "<br>".join(top_keywords)
        showlegend = False
        if pos == 1 and not added_legends["상승"]:
            showlegend = True
            added_legends["상승"] = True
        elif pos == -1 and not added_legends["하락"]:
            showlegend = True
            added_legends["하락"] = True

        marker_symbol = 'triangle-up' if pos == 1 else 'triangle-down'
        marker_color = 'red' if pos == 1 else 'blue'
        marker_name = '변곡점 (상승)' if pos == 1 else '변곡점 (하락)'

        fig.add_trace(go.Scatter( x=[date], y=[df['환율'].loc[date]], mode='markers', marker=dict(symbol=marker_symbol, color=marker_color, size=10),name=marker_name,hovertemplate=hover_text + '<extra></extra>' , showlegend=showlegend))

    # 서브프라임 모기지 사태
    annotation_date = df[df.index == '2007-07-02']

    fig.add_annotation(
        x=annotation_date.index[0],  # 주석을 추가할 날짜
        y=df.loc[annotation_date.index[0], '환율'],  # 주석을 추가할 y 값 (환율)
        text="서브프라임 모기지 사태",
        font_color = 'black',
        showarrow=True,
        arrowhead=1
    )

    # 글로벌 금융위기
    annotation_date = df[df.index == '2008-09-01']

    fig.add_annotation(
        x=annotation_date.index[0],  # 주석을 추가할 날짜
        y=df.loc[annotation_date.index[0], '환율'],  # 주석을 추가할 y 값 (환율)
        text="글로벌 금융위기",
        font_color = 'black',
        showarrow=True,
        arrowhead=1
    )

    # covid-19
    annotation_date = df[df.index == '2020-01-02']

    fig.add_annotation(
        x=annotation_date.index[0],  # 주석을 추가할 날짜
        y=df.loc[annotation_date.index[0], '환율'],  # 주석을 추가할 y 값 (환율)
        text="COVID-19",
        font_color = 'black',
        showarrow=True,
        arrowhead=1
    )

    # 러시아-우크라이나 전쟁
    annotation_date = df[df.index == '2022-02-24']

    fig.add_annotation(
        x=annotation_date.index[0],  # 주석을 추가할 날짜
        y=df.loc[annotation_date.index[0], '환율'],  # 주석을 추가할 y 값 (환율)
        text="러시아-우크라이나 전쟁 ",
        font_color = 'black',
        showarrow=True,
        arrowhead=1
    )

    # 마우스 오버 시 날짜와 환율 표시
    fig.update_layout(hovermode='x unified',xaxis_title='날짜',yaxis_title='환율',legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))

    # 그래프 출력
    col1.plotly_chart(fig)


    with col2:
        popover = st.popover(":grey_question:")
        multi = '''이 그래프는 100일 이동평균을 기준으로 한 환율 변곡점 분석 결과를 보여줍니다.
        변곡점 위에 마우스를 올리면 해당 날짜와 환율뿐만 아니라, 100일간의 뉴스 데이터를 분석하여 추출한 상위 10개의 키워드도 함께 확인할 수 있습니다.
        - 상승 변곡점 : 환율이 100일 이동평균을 상회하여 상승하는 시점 (빨간 삼각형)
        - 하락 변곡점 : 환율이 100일 이동평균을 하회하여 하락하는 시점 (파란 삼각형)
        '''
        popover.markdown(multi)

    ## 날짜 지정 키워드
    # CSV 파일 읽기
    df = pd.read_csv('최종.csv')
    keyword_df = pd.read_csv('환율(경제)_수정.csv')

    # 날짜 형식을 datetime으로 변환
    df['날짜'] = pd.to_datetime(df['날짜'])
    keyword_df['일자'] = pd.to_datetime(keyword_df['일자'])

    # 최소 및 최대 날짜
    min_date = keyword_df['일자'].min()
    max_date = keyword_df['일자'].max()

    col1, col2, col3 = st.columns([10,10,0.5])

    with col1:
        user_input_date_start = st.date_input("시작 날짜를 선택하세요", value=min_date, min_value=min_date, max_value=max_date)

    with col2:
        user_input_date_end = st.date_input("종료 날짜를 선택하세요", value=min_date, min_value=min_date, max_value=max_date)

    with col3:
        popover = st.popover(":grey_question:")
        multi = '''사용자가 직접 설정한 기간에 나타나는 상위 키워드 20개를 빈도수에 따라 보여줍니다.
        이 기능은 사용자가 특정 기간 동안 환율 변동 이유나 당시 이슈를 파악할 수 있도록 돕기 위해 마련되었습니다.
        '''
        popover.markdown(multi)

    # 사용자 입력 날짜를 datetime으로 변환
    user_input_date_start = pd.to_datetime(user_input_date_start)
    user_input_date_end = pd.to_datetime(user_input_date_end)

    # 날짜 범위에 해당하는 데이터 선택
    df_select = keyword_df.loc[(keyword_df['일자'] >= user_input_date_start) & (keyword_df['일자'] <= user_input_date_end)]

    # 모든 텍스트를 하나의 문자열로 결합
    all_text = ' '.join(df_select['특성추출(가중치순 상위 50개)'].tolist())

    # 쉼표(,)를 단위로 토큰화
    tokens = all_text.split(',')

    # 단어 빈도수 계산
    word_counts = Counter(tokens)

    # 단어 빈도수를 데이터프레임으로 변환
    word_freq_df = pd.DataFrame(word_counts.items(), columns=['단어', '빈도수'])

    # 빈도수 기준으로 정렬
    word_freq_df = word_freq_df.sort_values(by='빈도수', ascending=False)

    # 상위 20개 단어 추출 및 인덱스 1부터 20으로 설정
    top_20_words = word_freq_df.head(20).reset_index(drop=True)
    top_20_words.index += 1

    # HTML 테이블 생성
    html_table_keywords = top_20_words.to_html(index=False, justify='center')

    # CSS를 사용하여 가운데 정렬 및 너비 조정
    st.markdown(
        """
        <style>
        .keyword-table {
            width: 100%;
            margin: 0 auto;
        }
        .keyword-table table {
            width: 100%;
            table-layout: fixed;
        }
        .keyword-table th, .keyword-table td {
            text-align: center !important;
            width: 50%;
        }
        .keyword-table td {
            color: black;
        }
        .keyword-table td:contains('red') {
            color: red;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    # HTML 테이블 출력
    st.markdown(f'<div class="keyword-table">{html_table_keywords}</div>', unsafe_allow_html=True)


# information 탭
elif tabs == 'Information':
    # st.subheader(':blue[모델의 정확도]')
    # CSV 파일 읽기
    df = pd.read_csv('total.csv')

    # 데이터 전처리
    df['날짜'] = pd.to_datetime(df['날짜'])

    # 오차범위 계산
    df['예측환율_최소'] = np.round(df['예측환율'] - 1.96 * np.std(df['예측환율']), 2)
    df['예측환율_최대'] = np.round(df['예측환율'] + 1.96 * np.std(df['예측환율']), 2)

    # 실제 환율 데이터가 있는 마지막 날짜 찾기
    last_actual_date = df.dropna(subset=['실제환율'])['날짜'].max()

    # 실제 환율 데이터가 있는 날짜까지만 필터링
    df_filtered = df[df['날짜'] <= last_actual_date]

    # 2023년 1월 1일 이후 데이터 필터링
    df_filtered = df_filtered[df_filtered['날짜'] >= '2015-01-01']

    # 제목
    st.title('예측 환율 정확도 그래프')

    # 그래프 생성
    fig = go.Figure()

    # 실제 환율 데이터 추가
    fig.add_trace(go.Scatter(x=df_filtered['날짜'],y=df_filtered['실제환율'],mode='lines',name='실제 환율',line=dict(color='red'),hovertemplate='실제환율:%{y}<extra></extra>'))

    # 예측 환율 데이터 추가
    fig.add_trace(go.Scatter(x=df_filtered['날짜'],y=df_filtered['예측환율'],mode='lines',name='예측 환율',line=dict(color='green'),hovertemplate='예측환율:%{y}<extra></extra>'))

    # 오차범위 추가
    fig.add_trace(go.Scatter(x=df_filtered['날짜'].tolist() + df_filtered['날짜'][::-1].tolist(),y=df_filtered['예측환율_최대'].tolist() + df_filtered['예측환율_최소'][::-1].tolist(),fill='toself',fillcolor='rgba(0,100,80,0.2)',line_color='rgba(255,255,255,0)',hoverinfo="skip",showlegend=False))

    # 그래프 레이아웃 설정
    fig.update_layout(xaxis_title='날짜',yaxis_title='환율',xaxis=dict(tickformat='%Y-%m-%d'), hovermode="x unified", hoverlabel_bgcolor="white",hoverlabel_font_size=12,hoverlabel_font_family="Rockwell")

    # 그래프 표시
    st.plotly_chart(fig, use_container_width=True)

    multi = '''저희 프로젝트에서는 2019년부터 현재까지의 다양한 금융 및 경제 지표를 활용하여 LSTM(Long Short-Term Memory) 모델을 구축했습니다.
    데이터에는 다음과 같은 정보가 포함됩니다
    <br>- 주요 환율: EUR/KRW, GBP/KRW, JPY/KRW, EUR/USD, GBP/USD, JPY/USD
    <br>- 주요 주식 지수: 다우존스, S&P 500, 나스닥, FTSE, DAX, CAC, Nikkei
    <br>- 기타 지표: 미국 달러 지수, 미국 국채 금리(6개월, 1년, 3년, 5년, 10년), 금, 은, 천연가스, WTI, KOSPI 지수

    <br>LSTM 모델은 기존의 순환 신경망(RNN)에서 발생할 수 있는 장기 의존성 문제를 효과적으로 해결하는 데 초점을 맞추고 있습니다. RNN은 데이터의 순서를 고려하여 정보를 처리하지만, 장기적인 의존성을 다루는 데 어려움이 있었고, 그로 인해 긴 시계열 데이터에서 정확한 예측이 어려웠습니다. LSTM은 이러한 문제를 해결하기 위해 고안된 모델로, 장기적인 패턴을 기억하고, 중요한 정보는 유지하며 불필요한 정보는 잊어버리는 능력이 탁월합니다.
    <br>저희 환율 예측 모델은 이러한 LSTM의 장점 덕분에 2019년도부터 현재까지의 장기간 데이터를 효과적으로 처리하고 분석할 수 있습니다.
    <br>실제로, 예측값이 실제값과 거의 일치하는 것을 그래프를 통해 확인할 수 있습니다. 이는 LSTM 모델의 강력한 장기 기억 능력 덕분에 가능한 결과입니다.
    <br>이러한 정밀한 예측은 고객들에게 보다 신뢰할 수 있는 정보를 제공하며, 실제 금융 시장의 변동성을 더 잘 이해하고 대응할 수 있는 기회를 제공합니다.

    <br>**<span style="font-size: 0.8em;">본 페이지에서 제공되는 정보는 참고용이며, 정보의 정확성, 완전성, 신뢰성에 대해 보장하지 않습니다.
    정보의 오타나 발행 지체 등으로 인해 발생할 수 있는 어떠한 손해에 대해서도 책임을 지지 않습니다. 모든 판단과 결정은 본인의 몫이며, 이에 대한 책임은 본인에게 있습니다.**'''

    st.markdown(
        """
        <style>
        .highlight-text {
            background-color: #5c595a;
            color: white;
            padding: 20px;
            border-radius: 10px;
            border: 2px solid #fff;
            font-family: 'Arial', sans-serif;
            line-height: 1.6;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    st.markdown(
        f"""
        <div class="highlight-text">
            {multi}
        </div>
        """,
        unsafe_allow_html=True
    )